In [1]:
import scrap_dec

In [2]:
elemedecat = scrap_dec.PageElem(
    adresse = "https://www.decathlontravel.com/voyages/sport-rando-nature?gclid=Cj0KCQjw8eOLBhC1ARIsAOzx5cGfJP_HygFO_qlzeJG0KARJN657Ecyd7a3bGXh3OdxAGVnE14RFuREaAmPdEALw_wcB",
    bouton_accept = '//*[@id="didomi-notice-agree-button"]',
    bouton_affichage = '//*[@id="travel-search-engine-results"]/div[3]/a'
)

In [4]:
elemedecat.open_site()

In [5]:
elemedecat.affichage_all_item()

Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
